In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
'''
state estimation
x(k+1) = sqrt(5+x(k)) + w(k)
y(k) = x(k)^3 + v(k)
'''

SigmaW = 1 # Process noise covariance
SigmaV = 2 # Sensor noise covariance
maxIter = 40


# Define size of variables in model
Nx = 1     # state  = 1x1 scalar
Nxa = 3    # augmented state has also w(k) and v(k) contributions
Nz = 1     # output = 1x1 scalar

# constants for the SPKF algorithm. 
Wmx = np.zeros(2*Nxa+1)
h = np.sqrt(3)
Wmx[0] = (h**2-Nxa)/(h**2)
Wmx[1:] = 1/(2*h**2) 
Wcx=Wmx

xtrue = 2 + np.random.rand(1)   # Initialize true system initial state
xhat = 2                        # Initialize Kalman filter initial estimate
SigmaX = 1                      # Initialize Kalman filter covariance
u = 0                           # Unknown initial driving input: assume zero

# % Reserve storage for variables we might want to plot/evaluate
xstore = [xtrue.item()]
xhatstore = []
SigmaXstore = []

for k in range(maxIter):
    pass

In [ ]:
for k = 1:maxIter,
  % SPKF Step 1: State estimate time update
  % 1a: Calculate augmented state estimate, including ...
  xhata = [xhat; 0; 0]; % process and sensor noise mean
  % 1b: Get desired Cholesky factor
  Pxa = blkdiag(SigmaX,SigmaW,SigmaV);
  sPxa = chol(Pxa,'lower');   
  % 1c: Calculate sigma points (strange indexing of xhat to avoid
  % "repmat" call, which is very inefficient in Matlab)
  X = xhata(:,ones([1 2*Nxa+1])) + h*[zeros([Nxa 1]), sPxa, -sPxa];
  % 1d: Calculate state equation for every element
  % Hard-code equation here for efficiency
  Xx = sqrt(5+X(1,:)) + X(2,:);
  xhat = Xx*repWmxz';

  % SPKF Step 2: Covariance of prediction
  Xs = (Xx(:,2:end) - xhat(:,ones([1 2*Nxa])))*sqrt(Wcx(2));
  Xs1 = Xx(:,1) - xhat;
  SigmaX = Xs*Xs' + Wcx(1)*Xs1*Xs1';

  % [Implied operation of system in background, with
  w = chol(SigmaW)'*randn(1);
  v = chol(SigmaV)'*randn(1);
  zmeas = xtrue^3 + v;  % present z is based on present x and v
  xtrue = sqrt(5+xtrue) + w;  % future x is based on present x and w
  xstore(:,k+1) = xtrue;

  % SPKF Step 3: Create output estimate
  % Hard-code equation here for efficiency
  Z = Xx.^3 + X(3,:);
  zhat = Z*repWmxz';

  % SPKF Step 4: Estimator gain matrix
  Zs = (Z(:,2:end) - zhat*ones([1 2*Nxa])) * sqrt(Wcx(2));
  Zs1 = Z(:,1) - zhat;
  SigmaXZ = Xs*Zs' + Wcx(1)*Xs1*Zs1';
  SigmaZ  = Zs*Zs' + Wcx(1)*Zs1*Zs1';
  Lx = SigmaXZ/SigmaZ;

  % SPKF Step 5: Measurement state update
  xhat = xhat + Lx*(zmeas-zhat); % update estimate if not too rare

  % SPKF Step 6: Measurement covariance update
  SigmaX = SigmaX - Lx*SigmaZ*Lx'; 

  % [Store information for evaluation/plotting purposes]
  xhatstore(:,k) = xhat;
  SigmaXstore(:,k) = SigmaX(:);
end

subplot(1,2,1);
plot(0:maxIter-1,xstore(1:maxIter),'k-',0:maxIter-1,xhatstore,'b--', ...
  0:maxIter-1,xhatstore+3*sqrt(SigmaXstore),'m-.',...
  0:maxIter-1,xhatstore-3*sqrt(SigmaXstore),'m-.'); grid;
legend('true','estimate','bounds');
title('Sigma-point Kalman filter in action');
xlabel('Iteration'); ylabel('State');

subplot(1,2,2)
estErr = xstore(1:maxIter)-xhatstore; 
bounds = 3*sqrt(SigmaXstore);
plot(0:maxIter-1,estErr,'b-',0:maxIter-1, bounds,'m--',0:maxIter-1,-bounds,'m--');
grid; legend('Error','bounds',0);
title('SPKF Error with bounds');
xlabel('Iteration'); ylabel('Estimation Error');

In [ ]:
% Compute rms error
sqrt(mean(estErr.^2))

In [ ]:
% Compute fraction of estimates outside of 3-sigma bounds
length(find(abs(estErr)>bounds))/maxIter

In [ ]:
max(abs(estErr))